# Importing your own neural net

Verifying the example neural net was all well and good, but you probably want to verify your own neural net now. In this tutorial; we show you how to import the parameters for the feed-forward net in the introduction.

In [1]:
using MIPVerify
using Gurobi
using MAT
using JuMP
# export batch_processing_helper

We'll download a `.mat` file containing the parameters of the sample neural net, containing two layers (exported from `tensorflow`). 

In [2]:

model_path = joinpath("examples", "repaired_model_1.mat")
param_dict = matread("repaired_model_1.mat")
# param_dict = Base.download("https://github.com/vtjeng/MIPVerify_data/raw/master/weights/mnist/n1.mat") |> matread

Dict{String, Any} with 10 entries:
  "StatefulPartitionedCall… => Float32[-10.5 10.5]
  "StatefulPartitionedCall… => 0.709411
  "StatefulPartitionedCall… => Float32[-3.48292 -1.47885 … 0.446348 0.0]
  "StatefulPartitionedCall… => Float32[-0.354549 0.125176 … 0.0290697 -0.296804…
  "StatefulPartitionedCall… => Float32[0.045004 -0.171352 … -0.00617901 -0.0185…
  "StatefulPartitionedCall… => Float32[0.358638 -0.228793 … -0.280022 -0.125585…
  "StatefulPartitionedCall… => Float32[3.54272 -2.84389 … 3.69962 0.0501375; 3.…
  "StatefulPartitionedCall… => Float32[1.0 -1.0]
  "StatefulPartitionedCall… => Float32[-0.169738; -0.278756; … ; 0.00167305; 0.…
  "StatefulPartitionedCall… => Float32[-0.763657 -0.396411 … 0.146789 -0.63132]

In [3]:
dataset = matread("repair_data.mat")
x_adv = dataset["x_train"]
x_adv = round.(x_adv; digits=5)

25×40 Matrix{Float32}:
 1.67831  0.43944   0.23773  0.69642  1.70423  …  -0.95936   0.80634  1.09926
 1.204    0.82401  -0.30249  0.33595  1.25381      0.64304  -0.01957  0.52487
 0.43006  1.14952  -0.2984   0.30401  0.50807      0.97357  -0.11705  0.30053
 1.53349  0.44824   0.13504  0.65892  1.56156     -0.31463   0.64122  0.98883
 0.50095  1.05526  -0.37485  0.3943   0.56496      0.9987   -0.13765  0.39221
 1.67218  0.16923   0.2561   0.75616  1.67949  …  -0.95433   0.85573  1.08884
 0.727    0.97985  -0.27017  0.29498  0.78643      0.90822  -0.06625  0.35679
 0.87974  1.19853  -0.18941  0.2415   0.95806      0.79134   0.01217  0.43875
 1.6688   0.45955   0.26995  0.71032  1.69441     -0.83745   0.82711  1.13191
 1.46049  0.70588  -0.01628  0.50056  1.49954     -0.07081   0.3969   0.83533
 ⋮                                             ⋱                      
 1.38234  0.74986  -0.02407  0.42208  1.42578      0.30622   0.35864  0.77213
 0.76183  1.07663  -0.34318  0.19982  0.82621   

## Layers

Let's begin by importing the parameters for the first fully connected layer, which has 784 inputs (corresponding to a flattened 28x28 image) and 40 outputs.

### Basic Approach

We begin with a basic approach where we extract the weights and the biases of the fully connected layer seperately.

In [4]:
fc1_weight = param_dict["StatefulPartitionedCall/sequential/dense/MatMul/ReadVariableOp:0"]
fc1_bias = param_dict["StatefulPartitionedCall/sequential/dense/BiasAdd/ReadVariableOp:0"]
fc1 = Linear(fc1_weight, dropdims(fc1_bias, dims=1))



Linear(40 -> 32)

In [5]:
fc2_weight = param_dict["StatefulPartitionedCall/sequential/dense_1/MatMul/ReadVariableOp:0"]
fc2_bias = param_dict["StatefulPartitionedCall/sequential/dense_1/BiasAdd/ReadVariableOp:0"]
fc2 = Linear(fc2_weight, dropdims(fc2_bias, dims=1))

Linear(32 -> 32)

We group the weights and biases in a `Linear`.

_(NB: We have to flatten the bias layer using `dropdims` since `Linear` expects a 1-D array for the bias.)_

In [6]:


fc3_weight = param_dict["StatefulPartitionedCall/sequential/dense_2/MatMul/ReadVariableOp:0"]
fc3_bias = param_dict["StatefulPartitionedCall/sequential/dense_2/BiasAdd/ReadVariableOp:0"]
fc3 = Linear(fc3_weight, dropdims(fc3_bias, dims=1))



Linear(32 -> 32)

In [7]:
fc4_weight = param_dict["StatefulPartitionedCall/sequential/dense_3/MatMul/ReadVariableOp:0"]
fc4_bias = param_dict["StatefulPartitionedCall/sequential/dense_3/BiasAdd/ReadVariableOp:0"]
fc4 = Linear(fc4_weight, [fc4_bias])



Linear(32 -> 1)

In [8]:
fc5_weight = param_dict["StatefulPartitionedCall/sequential/output/MatMul/ReadVariableOp:0"]
fc5_bias = param_dict["StatefulPartitionedCall/sequential/output/BiasAdd/ReadVariableOp:0"]
fc5 = Linear(fc5_weight, dropdims(fc5_bias, dims=1))

Linear(1 -> 2)

As a sanity check, you can verify that the parameters we get from both methods are equal.

## Composing the network

We now put the entire network together. We need to start by flattening the input since the input images are provided as a 4-dimensional tensor.

In [9]:
n1 = Sequential([
        Flatten(1),
        fc1,
        # you can always use interval arithmetic for the first layer
        ReLU(),
        fc2,
        ReLU(),
        fc3,
        ReLU(),
        fc4,
        fc5,
    ], "bounding_const_model_repaired_1.n1")

sequential net bounding_const_model_repaired_1.n1
  (1) Flatten(): flattens 1 dimensional input, with dimensions permuted according to the order [1]
  (2) Linear(40 -> 32)
  (3) ReLU()
  (4) Linear(32 -> 32)
  (5) ReLU()
  (6) Linear(32 -> 32)
  (7) ReLU()
  (8) Linear(32 -> 1)
  (9) Linear(1 -> 2)


In [10]:
# get the dim of x_adv
num_samples = size(x_adv)[1]
report = []
new_adv_samples = []
for i in 1:num_samples
    d = MIPVerify.find_adversarial_example(
        n1, 
        x_adv[i,:], 
        1, 
        Gurobi.Optimizer, 
        Dict("OutputFlag" => 0),
        tightening_algorithm = lp,
        norm_order = Inf,
        pp=MIPVerify.LInfNormBoundedPerturbationFamily(1)
    )
    # println(d[:SolveStatus]== INFEASIBLE_OR_UNBOUNDED) 
    report = [report; d]
    # println(d[:SolveStatus])
    if d[:SolveStatus] != INFEASIBLE_OR_UNBOUNDED
        println("feasible")
        new_adv_samples = [new_adv_samples; JuMP.value.(d[:PerturbedInput])]
    end
end
new_adv_samples = reshape(new_adv_samples, (size(x_adv)[2], convert(Int64, floor.(0.01+length(new_adv_samples)/size(x_adv)[2]))))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-15
[notice | MIPVerify]: Attempting to find adversarial example. Neural net predicted label is 2, target labels are [1]


[notice | MIPVerify]: Determining upper and lower bounds for the input to each non-linear unit.


  Calculating upper bounds:   6%|█▌                     |  ETA: 0:01:55

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-15
Set parameter TimeLimit to value 20
Set parameter TimeLimit to value 20


  Calculating upper bounds: 100%|███████████████████████| Time: 0:00:08


...

  Calculating lower bounds:  62%|██████████████▍        |  ETA: 0:00:00

  Calculating lower bounds:  84%|███████████████████▍   |  ETA: 0:00:00

  Calculating lower bounds: 100%|███████████████████████| Time: 0:00:00


Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-15
feasible


MethodError: MethodError: First argument to `convert` must be a Type, got Inf

In [18]:
new_adv_samples = new_adv_samples'

25×40 adjoint(::Matrix{Any}) with eltype Any:
 1.0      0.0        1.0        0.0       …  0.0      0.0       1.0
 0.76786  0.2672     0.0        0.0          0.08623  0.53724   1.0
 0.13166  1.0        0.0        0.60241      1.0      0.18135   0.59893
 1.0      0.0        0.0        0.857677     0.0      0.0       1.0
 0.1261   1.0        0.0        0.432576     0.62385  0.0       0.708494
 1.0      0.0        0.0        1.0       …  0.0      0.0       1.0
 0.45683  1.0        0.0        0.56515      0.63805  0.20392   0.62696
 1.0      1.0        0.0        0.66402      1.0      0.358191  0.86127
 1.0      0.0        0.0        0.900028     0.0      0.0       0.298313
 1.0      0.0328199  0.0        1.0          0.0      0.0       1.0
 ⋮                                        ⋱                     
 1.0      0.0893599  0.0        0.129886     0.0      0.0       0.555381
 0.41865  1.0        0.0        0.543        0.41422  0.17171   0.71247
 0.49176  1.0        0.0        0.61038   

In [25]:
# Report time taken for adversarial verification
println("Time taken for adversarial verification: ", sum([report[i][:TotalTime] for i in 1:length(report)]))


Time taken for adversarial verification: 91.35036956799999


In [ ]:
# store the new adersarial num_samples
matwrite("new_adv_samples_repaired_1.mat", Dict("new_adv_samples" => new_adv_samples))

## Create MIPVerify net

There we go! Now it's your turn to try to verify your own neural network.

In [35]:
# input = [1.67830884;  0.43943787;  0.23773319;  0.69642472;  1.70423186;
# 0.36528784;  0.28762001;  0.75129396;  1.72261453;  0.20819041;
# 0.34219459;  0.80685776;  1.72969389;  0.00710564;  0.3986893 ;
# 0.85964334;  1.72499692; -0.16255961;  0.45739096;  0.90201074;
# 1.7116487 ; -0.28321046;  0.45739096;  0.94229448;  1.69120371;
# -0.39003676;  0.58705908;  0.97771639;  1.66370773; -0.54085034;
# 0.65706277;  1.02980745;  1.62546659; -0.73188078;  0.73023951;
# 1.07148027;  1.57448554; -0.95935792;  0.80633789;  1.09926224]
input = [ 1.67830884,  0.43943787,  0.23773319,  0.69642472,  1.70423186,
0.36528784,  0.28762001,  0.75129396,  1.72261453,  0.20819041,
0.34219459,  0.80685776,  1.72969389,  0.00710564,  0.3986893 ,
0.85964334,  1.72499692, -0.16255961,  0.45739096,  0.90201074,
1.7116487 , -0.28321046,  0.45739096,  0.94229448,  1.69120371,
-0.39003676,  0.58705908,  0.97771639,  1.66370773, -0.54085034,
0.65706277,  1.02980745,  1.62546659, -0.73188078,  0.73023951,
1.07148027,  1.57448554, -0.95935792,  0.80633789,  1.09926224]
input |> n1

2-element Vector{Float64}:
 -0.5011958860224333
  0.5011958860224333

In [36]:
input = round.(input; digits=3)

40-element Vector{Float64}:
  1.678
  0.439
  0.238
  0.696
  1.704
  0.365
  0.288
  0.751
  1.723
  0.208
  ⋮
  1.03
  1.625
 -0.732
  0.73
  1.071
  1.574
 -0.959
  0.806
  1.099

In [34]:
# MIPVerify.set_log_level!("info")
d = MIPVerify.find_adversarial_example(
    n1, 
    input, 
    1, 
    Gurobi.Optimizer, 
    Dict("OutputFlag" => 0),
    tightening_algorithm = lp,
    norm_order = Inf,
    pp=MIPVerify.LInfNormBoundedPerturbationFamily(1)
)

UndefVarError: UndefVarError: input not defined

In [21]:
# concatenate the perturbed input with the original input
# to get the adversarial example
adv_example = hcat(input, purturbed_inp)


40×2 Matrix{Float64}:
  1.678  1.0
  0.439  0.0
  0.238  0.718076
  0.696  1.0
  1.704  1.0
  0.365  0.0
  0.288  0.0
  0.751  0.0
  1.723  1.0
  0.208  1.0
  ⋮      
  1.03   1.0
  1.625  0.666
 -0.732  0.0
  0.73   1.0
  1.071  1.0
  1.574  1.0
 -0.959  0.0
  0.806  1.0
  1.099  1.0

In [33]:
report[3]

Dict{Any, Any} with 11 entries:
  :TargetIndexes      => [1]
  :SolveTime          => 0.556749
  :TotalTime          => 1.80739
  :Perturbation       => VariableRef[_[1], _[2], _[3], _[4], _[5], _[6], _[7], …
  :PerturbedInput     => VariableRef[_[41], _[42], _[43], _[44], _[45], _[46], …
  :TighteningApproach => "lp"
  :PerturbationFamily => linf-norm-bounded-1
  :SolveStatus        => OPTIMAL
  :Model              => A JuMP Model…
  :Output             => AffExpr[-0.16973836719989777 _[205] - 0.27875575423240…
  :PredictedIndex     => 2

In [43]:
perturbed_sample_image = JuMP.value.(d[:PerturbedInput])
out = perturbed_sample_image|>n1
out[1]

0.02726411541655871